# Notebook 3: Reinforcement Learning

**Authors:** Course material from Shimon Whiteson. Introduction to reinforcement learning, 2019 (https://github.com/mlss-skoltech/tutorials_week2).

Adapted for the ML4Q-retreat 2022 by Alexander Gresch

In [ ]:
!pip install gym[toy_text]

In [ ]:
import gym
import numpy as np
import matplotlib.pyplot as plt
from time import sleep
from IPython.display import clear_output
from IPython import display as ipythondisplay

import random
import warnings
from collections import defaultdict

In [ ]:
%matplotlib inline

## Dyna-Q: RL with action values

In this exercise you will implement **Dyna-Q**, which is a model-based Q-planning algorithm (planning with action values).

## Cliff walking Environment

In this exercise we will work with the Cliff walking environment from OpenAI's gym library.  
Make yourself familiar with the environment:
- https://www.gymlibrary.dev/environments/toy_text/cliff_walking/

The environment also provides some useful attributes:
- ```env.nS```: the number of states. The env sets up a grid of shape (4x12) s.t. there are 48 distinct states.
- ```env.nA```: the number of actions. The agent can decide to move to adjacent grid places, but not vertically. This makes up 4 possible movements.
    
Each time we start anew, we have to reset the environment via

    state = env.reset()

Note that this yields an intial state that is presented to the agent.
Upon choosing an action, the environment provides the agent with

    
    state, reward, done, info = env.step(action)
    
where ```done``` is a boolean that flags whether we have reached the terminal state. ```info``` may contain further information about the environment (which is not presented to the agent, however).

Your task (and the task of RL in general) is to tune an algorithm that picks the sequence of actions that maximize the (discounted) return.

In [ ]:
# let's initialize the environment
env = gym.make("CliffWalking-v0").env

Print the observation space and action space of the environment to see if it matches the description above:

In [ ]:
print(env.observation_space)
print(env.action_space)

## Policies

In our setup, policies are functions that take two arguments, ```env``` and ```state```, and return an action based on that state:

```
def my_policy(env, state):
    action = ...
    return action
```

Below we implemented a function that runs one rollout of a given policy:

In [ ]:
def rollout(env, policy, max_steps=100, render=False):
    state = env.reset()
    total_reward = 0.
    num_steps = 0
    done = False
    while not done:
        if render:
            env.render()
            clear_output(wait=True)
            
        action = policy(env, state)
        state, reward, done, info = env.step(action)
        total_reward += reward
        
        num_steps += 1
        done = done or num_steps > max_steps
        
        if render:
            sleep(0.4)
    
    if render:
        env.render()
    return total_reward

And another function that runs multiple rollouts of a given policy and averages the total rewards:

In [ ]:
def evaluate(env, policy, num_rollouts=100, max_steps=100):
    return sum(rollout(env, policy, max_steps=max_steps) for _ in range(num_rollouts)) / num_rollouts

### Random policy
The random policy selects random actions and ignores the current state.  
Implement this policy by selecting the next action at random, regardless of the given state:

In [ ]:
def random_policy(env, state):
    ################################
    ### TODO: ###
    ################################
    # return a valid action at random
    # action = ...
    ################################
    return action

In [ ]:
# have a look how the random policy performs over 20 steps
total_reward = rollout(env, random_policy, max_steps=20, render=True)
print('\nTotal reward:', total_reward)

Unfortunately, the rendering does not work with google colab :(


In [ ]:
# without rendering, it is very fast to evaluate the average performance of the same policy over 20 steps as
print('Average total reward:', evaluate(env, random_policy, max_steps=20))

## Implement Dyna-Q
Implement the Tabular Dyna-Q algorithm described in Chapter 8.2 of [Reinforcement Learning: An Introduction](http://incompleteideas.net/book/the-book-2nd.html) by Sutton and Barto, p. 164. (You do not need to look up the original source. The book is nevertheless a very valuable ressource if you want to dive deeper into the field of RL.)

![](http://incompleteideas.net/book/first/ebook/pseudotmp18.png)

Some remarks:
- we initialize all $Q(s,a) = 0$
- in practice, $Q$ will be a dictionary with keys $s$ and corresponding lists as items for all possible actions $a$. This means that $Q[s][a] \equiv Q(s,a)$
- with Model(s,a), we simply store previously observed (state,action,new_state,reward) combinations such that we can reuse them later to train the agent more efficiently. This is done for step (f)
- instead of "forever", we preset a fixed number of episodes instead. Your time is valuable, I understand this :)
- in step (f), we only use Model(s,a) to further adjust the Q-values. We do NOT alter the current state of the environment

Dyna-Q uses the an $\epsilon$-greedy action selection and Q update formula (in step (d)).
Implement those two first. I also provide you with the function argmax_random that selects randomly from those indices for which x is maximum. This prevents breaking ties for multiple maxima at the same time.

In [ ]:
def argmax_random(x):
    return np.random.choice(np.flatnonzero(x == x.max()))

def epsilon_greedy_action(env, Q, epsilon, state):
    ##################################################
    ### TODO: ###
    ##################################################
    # with probability epsilon, follow a random policy,
    # otherwise, pick one of the argmax of Q[state]
    # action = ...
    ##################################################
    return action

def update_Q(Q, old_state, new_state, action, reward, alpha, gamma=0.95):
    ##################################################
    ### TODO: ###
    ##################################################
    # implement step (d)
    # Q[...] = ...
    ##################################################
    return Q

Now implement the main part of the algorithm:

In [ ]:
def train_dyna_Q(env, num_episodes, n=0, alpha=0.1, epsilon=0.1, silent=False):
    """Estimate optimal action-value function using the Dyna-Q algorithm.
    
    Args:
        env: The environment to train on.
        num_episodes: Number of episodes to run.
        n: The number of planning steps in Dyna-Q
        alpha: Step size for the action-value update.
        epsilon: Parameter for the epsilon greedy action selection.
    
    Returns:
        Q: The finished Q-table.
        num_steps: Array containing number of steps per episode.
        episode_rewards: Array containing the sum of rewards of the episode.
    """
    
    # initialize Dyna-Q variables
    Q = defaultdict(lambda: np.zeros(env.action_space.n)) # this sets all action values to zero
    model = {}
    
    num_steps = []
    episode_rewards = []
    
    for episode in range(1, num_episodes + 1):
        if not silent:
            print("\rEpisode {}/{}".format(episode, num_episodes), flush=True, end='')
        
        state = env.reset() # this already accomplishes step (a)
        done = False
        steps = 0
        cumulated_reward = 0
        while not done:
            # step (b) to (f)
            ##################################################
            ### TODO: ###
            ##################################################
            # implement step (b)
            # ...
            # implement step (c)
            # ...
            # step (d)
            # ...
            # step (e)
            # ...
            # finally, step (f). Take n samples stored in model at random to update Q further
            # ...
            ##################################################
            # we also keep track of the total number of steps and all rewards gained so far
            steps += 1
            cumulated_reward += reward
            
        
        num_steps.append(steps)
        episode_rewards.append(cumulated_reward)
    
    return Q, np.array(num_steps), np.array(episode_rewards)

Let's test your implementation! Choose at least 200 episodes with a small number of replays $n$:

In [ ]:
# change these to your liking
num_episodes = 200
num_replays  = 5

env = gym.make("CliffWalking-v0").env
Q, num_steps, rewards = train_dyna_Q(env, num_episodes, num_replays)

We can plot how num_steps and rewards changes during training of the agent. To make the curves a bit smoother, we use an moving average over some recent episodes. The black dotted line corresponds to the optimal policy.

In [ ]:
def moving_average(x, window_size):
    return np.convolve(x, np.ones(window_size), 'valid') / window_size

In [ ]:
plt.plot(num_steps,alpha=0.5)
plt.plot(moving_average(num_steps,10),"r-")
plt.hlines(13,0,num_episodes,colors="black",linestyles="dashed")
plt.ylim(12,50)

We also plot the negative reward (again with a moving average)

In [ ]:
plt.plot(-rewards,alpha=0.5)
plt.plot(moving_average(-rewards,10),"r-")
plt.hlines(13,0,num_episodes,colors="black",linestyles="dashed")
plt.ylim(0,300)

After training, your agent should find the goal easily:

In [ ]:
state = env.reset()
for i in range(100):
    action = epsilon_greedy_action(env, Q, 0.3, state)
    state, reward, done, info = env.step(action)
    env.render()
    if done:
        print("Walk stopped after {} steps.".format(i+1))
        break
    sleep(40e-3)

The optimal number of steps is 13. Why is your agent performing worse?

**Answer:** ...

## Key parameters in Dyna-Q

Now you will explore the role of the key parameters $n$ and $\alpha$. 

#### Exploring $n$
In particular, plot the two curves from above averaged over 10 runs for $n \in \{0,5,50\}$. Keep the number of episodes fixed to 50

In [ ]:
%%time
# measures the execution time
# first collect the data for plotting
##################################################
### TODO: ###
##################################################
# define various values for n and train the agent 10 independent times on 50 episodes each.
# Keep track on the average num_steps and the average cumulative reward over the 10 runs.
# no_eps = ... counts the average number of steps averaged over 10 runs per n-value and episode
# no_rewards = ... counts the average cumulative reward averaged over 10 runs per n-value and episode
##################################################

In [ ]:
# then plot the data for the number of steps
plt.figure()
for i,n in enumerate(n_vals):
    plt.plot(np.arange(1,51),no_eps[i],label="n = {} ".format(n))
plt.hlines(13,1,50,colors="black",linestyles="dashed")
plt.legend()
plt.ylim(10,300)
plt.xlim(1,50)

In [ ]:
# then plot the data for the cumulative rewards
plt.figure()
for i,n in enumerate(n_vals):
    plt.plot(np.arange(1,51),no_rewards[i],label="n = {} ".format(n))
plt.hlines(-13,1,50,colors="black",linestyles="dashed")
plt.legend()
plt.ylim(-300,-10)
plt.xlim(1,50)

#### Exploring $\alpha$
Now also plot the learning curves averaged over 10 runs for $\alpha \in \{0.0625, 0.25, 0.5, 1.0\}$. Keep $n=5$ as a result of the previous plot

In [ ]:
%%time
# measures the execution time
# first collect the data for plotting
##################################################
### TODO: ###
##################################################
# repeat the previous task with varying alpha values
# no_eps_alpha = ...
# no_rewards_alpha = ...
##################################################

In [ ]:
# then plot the data
plt.figure()
for i,alpha in enumerate(alpha_vals):
    plt.plot(np.arange(1,51),no_eps_alpha[i],label="alpha = {} ".format(alpha))
plt.legend()
plt.hlines(13,1,50,colors="black",linestyles="dashed")
plt.xlim(1,51)
plt.ylim(10,60)
plt.show()

In [ ]:
# then plot the data for the cumulative rewards
plt.figure()
for i,n in enumerate(n_vals):
    plt.plot(np.arange(1,51),no_rewards_alpha[i],label="n = {} ".format(n))
plt.hlines(-13,1,50,colors="black",linestyles="dashed")
plt.legend()
plt.ylim(-300,-10)
plt.xlim(1,50)